In [2]:
# Import Library
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset

import pandas as pd
from PIL import Image
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

/opt/homebrew/anaconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/opt/homebrew/anaconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <F0D48035-EF9E-3141-9F63-566920E60D7C> /opt/homebrew/anaconda3/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <9DDA9B08-547F-32F9-87F1-796977059897> /opt/homebrew/anaconda3/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [3]:
# Apple GPU Availability Verfication
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
else:
    mps = True
    print("Apple GPU is available")

Apple GPU is available


In [4]:
# CustomImageDataset & DataLoader
torch.manual_seed(42)
epochs = 100
batch_size = 512

# Define Custom Image Dataset Class
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file):
        self.img_labels = pd.read_csv(annotations_file)
        self.transform = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            # transforms.Normalize(mean=[0.5], std=[0.5]),
            # transforms.Lambda(lambda x: (x + 1.0) / 2.0)
        ])

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_labels.iloc[idx, 0]
        image = Image.open(img_path)
        image = self.transform(image)
        label = self.img_labels.iloc[idx, 1]
        return image, label

# Call the CID object
csv_file = "train_data.csv"
cid = CustomImageDataset(csv_file)

# Define Dataloader
train_loader = DataLoader(
    cid,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
)

csv_file = "test_data.csv"
cid = CustomImageDataset(csv_file)

test_loader = DataLoader(
    cid,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
)

In [5]:
# Define CNN Architecture
class Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Classifier, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=2),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),  
            nn.Conv2d(128, 256, kernel_size=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [6]:
# Initialize Model, Loss Function, Optimizer and Scheduler
model = Classifier(num_classes=5)
criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.163, 0.158, 0.287, 0.193, 0.197]))
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True, min_lr=1e-6)

# Switch to GPU/CPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
criterion.to(device)


CrossEntropyLoss()

In [7]:
# Model Training & Validation
num_epochs = 20

train_losses = []
train_accuracies = []
valid_losses = []
valid_accuracies = []

true_label = []
pred_label = []

for epoch in range(1, num_epochs + 1):
    # Starting Training
    model.train()
    correct = 0
    total = 0
    loss = 0.0
    
    for step, (vector, label) in enumerate(train_loader):
        # Switch to GPU/CPU
        vector = vector.to(device)
        label = label.to(device)
        
        # Clear Gradient --> Prevent Accumulating Previous Gradient
        optimizer.zero_grad()
        
        # Forward Pass --> Generate Predicted Output(Max Probability Class) from CNN
        logits = model(vector)
        _, predicted = torch.max(logits.data, 1)
        
        # Backward Propagation & Update Parameter --> Optimizer To Fit CNN
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()
        
        # Accumulate Training Loss, # of Correct Prediction & # of Label 
        total += label.size(0)
        loss += loss.item()
        correct += (predicted == label).sum().item()
        
        # Save Results for Confusion Matrix
        true_label.extend(label.tolist())
        pred_label.extend(predicted.tolist())

        print(loss)

    # Average Training Loss & Calculate Training Accuracy
    epoch_train_loss = loss / len(train_loader)
    epoch_train_accuracy = 100 * correct / total
    
    # Print & Save Results for Plot
    train_losses.append(epoch_train_loss)
    train_accuracies.append(epoch_train_accuracy)
    print(f'Epoch [{epoch}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Train Accuracy: {epoch_train_accuracy:.2f}%')
    
    #----------------------------------------------------------------------------------------------------------------------
    
    # Start Validating
    model.eval()
    correct = 0
    total = 0
    valid_loss = 0.0

    with torch.no_grad():
        for vector, label in test_loader:
            # Switch to GPU/CPU
            vector = vector.to(device)
            label = label.to(device)
            
            # Forward Pass & Produce Output
            pred = model(vector)
            _, predicted = torch.max(pred.data, 1)

            # Accumulate Validation Loss, # of Correct Prediction, # of Labels
            total += label.size(0)
            loss = criterion(pred, label)
            valid_loss += loss.item()
            correct += (predicted == label).sum().item()
            
            # Save Results for Confusion Matrix
            true_label.extend(label.tolist())
            pred_label.extend(predicted.tolist())
    
    # Average Validation Loss & Calculate Validation Accuracy
    epoch_valid_loss = loss / len(test_loader)
    epoch_valid_accuracy = 100 * correct / total

    # Print & Save Results for Plot
    valid_losses.append(epoch_valid_loss)
    valid_accuracies.append(epoch_valid_accuracy)
    print(f'Epoch [{epoch}/{num_epochs}], Validation Loss: {epoch_valid_loss:.4f}, Validation Accuracy: {epoch_valid_accuracy:.2f}%')
   
    # Activate Scheduler
    scheduler.step(epoch_valid_loss)

tensor(3.8323, device='mps:0', grad_fn=<AddBackward0>)
tensor(3.8730, device='mps:0', grad_fn=<AddBackward0>)
tensor(3.2008, device='mps:0', grad_fn=<AddBackward0>)
tensor(3.5105, device='mps:0', grad_fn=<AddBackward0>)
tensor(3.0338, device='mps:0', grad_fn=<AddBackward0>)
tensor(3.0146, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.9143, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.9160, device='mps:0', grad_fn=<AddBackward0>)
tensor(3.0519, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.8937, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.8929, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.8140, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.8047, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.7941, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.7738, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.5972, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.7203, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.6509, device='mps:0', grad_fn=<AddBackward0>)
tensor(2.6

Epoch [3/20], Validation Loss: 0.1064, Validation Accuracy: 45.87%
tensor(1.8349, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.7103, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.7057, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.7042, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.8239, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.7729, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.6394, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.6644, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.6593, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.6631, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.7182, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.6471, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.7183, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.7127, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.6826, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.7501, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.5677, device='mps:0', grad_fn=<AddBackward0>

Epoch [6/20], Validation Loss: 0.1040, Validation Accuracy: 51.92%
tensor(1.0244, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0275, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0750, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0069, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.9450, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.9020, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0630, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.9379, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.9526, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.9764, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.8879, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.9514, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0247, device='mps:0', grad_fn=<AddBackward0>)
tensor(1.0163, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.9593, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.9122, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.9693, device='mps:0', grad_fn=<AddBackward0>

tensor(0.4008, device='mps:0', grad_fn=<AddBackward0>)
Epoch [9/20], Train Loss: 0.0083, Train Accuracy: 94.49%
Epoch [9/20], Validation Loss: 0.0987, Validation Accuracy: 58.41%
tensor(0.3625, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3992, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3869, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3583, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3843, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3987, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3938, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3803, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3922, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3423, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3809, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3766, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3621, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3805, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3851, device='mps:0', grad_fn=<AddBackward

tensor(0.3040, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3012, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3014, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2984, device='mps:0', grad_fn=<AddBackward0>)
Epoch [12/20], Train Loss: 0.0062, Train Accuracy: 97.75%
Epoch [12/20], Validation Loss: 0.0989, Validation Accuracy: 58.24%
tensor(0.3054, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3145, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3155, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3280, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2940, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3370, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3029, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2885, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2899, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2905, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2927, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3227, device='mps:0', grad_fn=<AddBackwa

tensor(0.3404, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2778, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2764, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3236, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2944, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2829, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2840, device='mps:0', grad_fn=<AddBackward0>)
Epoch [15/20], Train Loss: 0.0059, Train Accuracy: 98.06%
Epoch [15/20], Validation Loss: 0.0927, Validation Accuracy: 58.45%
tensor(0.3086, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2789, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2793, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2798, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3146, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2945, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2980, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3073, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3032, device='mps:0', grad_fn=<AddBackwa

tensor(0.2948, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3542, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2861, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2596, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2839, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2771, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2827, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3058, device='mps:0', grad_fn=<AddBackward0>)
Epoch [18/20], Train Loss: 0.0064, Train Accuracy: 98.05%
Epoch [18/20], Validation Loss: 0.1019, Validation Accuracy: 58.54%
tensor(0.2838, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3315, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2854, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2905, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.2581, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3283, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3004, device='mps:0', grad_fn=<AddBackward0>)
tensor(0.3061, device='mps:0', grad_fn=<AddBackwa

In [8]:
# Model Evaluation
print("Confusion Matrix:")
print(confusion_matrix(true_label, pred_label))

precision, recall, f1, _ = precision_recall_fscore_support(true_label, pred_label, average='micro')
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")


Confusion Matrix:
[[ 67741   4919   8601  11236   7764]
 [  4134  67056   8750   9013   8199]
 [  1548   1543 164911   3983   4294]
 [  5088   4143  10282  88260  11352]
 [  3043   3355  10728   8905  95552]]
Precision: 0.7870, Recall: 0.7870, F1-Score: 0.7870
